# Install, Paths and Parameters

In [1]:
# Requirements. Need to restart after installation (it sucks, I am sorry haha)
!pip install torch==1.7.1
!pip install torchvision==0.8.2
!pip install torchattacks

     |████████████████████████████████| 776.8 MB 17 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.7.1 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.7.1 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.7.1 which is incompatible.
     |████████████████████████████████| 12.8 MB 5.8 MB/s 
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
     |████████████████████████████████| 102 kB 4.6 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [48]:
import os
import pandas as pd
import numpy as np
import json
import torch
from typing import List, Callable
import random

from sklearn import preprocessing

# seed
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

In [4]:
# general path
DRIVE_PATH = '/content/drive/MyDrive/'
DL_PROJECT_PATH = os.path.join(DRIVE_PATH, 'DeepLearningProject')
ADV_PATH = os.path.join(DL_PROJECT_PATH, 'AdversarialAttacks')

# filenames and label path
ORG_LABEL_PATH = os.path.join(ADV_PATH,'ori_data/correct_labels_cl.txt')

# image paths
ORIGINAL_IMAGES_PATH = os.path.join(ADV_PATH,'ori_data/ImageNetClasses/')

In [43]:
def get_random_classes(number_of_classes: int = 50, min_rand_class: int = 0, max_rand_class: int = 999):
  return np.random.randint(low=min_rand_class, high=max_rand_class, size=(number_of_classes,))

def get_random_indexes(number_of_images: int = 50000, n_samples=1000):
  return np.random.choice(50000, 1000, replace=False)


NUM_CLASSES = 20
RANDOM_CLASSES = get_random_classes(number_of_classes=NUM_CLASSES)

# Import DINO
Official repo: https://github.com/facebookresearch/dino

In [6]:
# Load pretrained weights from PyTorch
device = 'cuda'
vits16 = torch.hub.load('facebookresearch/dino:main', 'dino_vits8').to(device)

Downloading: "https://github.com/facebookresearch/dino/archive/main.zip" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/dino/dino_deitsmall8_pretrain/dino_deitsmall8_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dino_deitsmall8_pretrain.pth


  0%|          | 0.00/82.7M [00:00<?, ?B/s]

In [45]:
# Hyperparameters
N_LAST_BLOCKS = 4 # Took from official repo
PATCH_SIZE=8

In [46]:
# Define and load pretrained weights for linear classifier on ImageNet
from torch import nn
class LinearClassifier(nn.Module):
    """Linear layer to train on top of frozen features"""
    def __init__(self, dim, num_labels=1000):
        super(LinearClassifier, self).__init__()
        self.num_labels = num_labels
        self.linear = nn.Linear(dim, num_labels) 
        self.linear.weight.data.normal_(mean=0.0, std=0.01)
        self.linear.bias.data.zero_()

    def forward(self, x):
        # flatten
        x = x.view(x.size(0), -1)

        # linear layer
        return self.linear(x)

linear_classifier = LinearClassifier(vits16.embed_dim * N_LAST_BLOCKS, num_labels=NUM_CLASSES)
linear_classifier = linear_classifier.cuda()

# Load data

In [55]:
# Custom loader
from torch.utils.data import Dataset
from torchvision.io import read_image
import traceback
from PIL import Image

# MAP CLASSES TO [0, NUM_CLASSES]
le = preprocessing.LabelEncoder()
le.fit([i for i in RANDOM_CLASSES])

class ImageDataset(Dataset):
  def __init__(self, img_folder: str, file_name: str, transform: callable, class_subset: List[int] = None, index_subset: List[int] = None):
    super().__init__()
    self.transform=transform
    self.img_folder=img_folder
    self.data = self.create_df(file_name)
    self.class_subset = class_subset
    if self.class_subset is None:
      self.data_subset = self.data
    else:
      self.data_subset = self.data[self.data['label'].isin(self.class_subset)]

    self.data_subset['label'] = le.transform(self.data_subset['label'])
  
  def create_df(self, file_name: str):
    df = pd.read_csv(file_name, sep=" ", header=None)
    df.columns=['file', 'label']
    return df
    
  def __len__(self):
    return len(self.data_subset)
  
  def __getitem__(self, index):
    img = Image.open(os.path.join(self.img_folder,self.data_subset['file'].iloc[index]))
    img = img.convert('RGB')

    img=self.transform(img)
    target=self.data_subset['label'].iloc[index]

    return img,target,self.data_subset['file'].iloc[index]


In [56]:
from torchvision import transforms as pth_transforms

BATCH_SIZE = 8

# Create loader
# Taken from official repo: https://github.com/facebookresearch/dino/blob/main/eval_linear.py
loader_transform = pth_transforms.Compose([
    pth_transforms.Resize(256, interpolation=3),
    pth_transforms.CenterCrop(224),
    pth_transforms.ToTensor()
])

dataset = ImageDataset(img_folder = ORIGINAL_IMAGES_PATH,
                           file_name = ORG_LABEL_PATH,
                           transform=loader_transform,
                           class_subset=RANDOM_CLASSES)

train_set, val_set = torch.utils.data.random_split(dataset, [800, 200])

train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=BATCH_SIZE,
    num_workers=1,
    pin_memory=True,
)

test_loader = torch.utils.data.DataLoader(
    val_set,
    batch_size=BATCH_SIZE,
    num_workers=1,
    pin_memory=True,
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [57]:
BATCH_SIZE

8

In [58]:
class ViTModel(torch.nn.Module):
    def __init__(self, vits16, linear_layer):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(ViTModel, self).__init__()
        self.vits16=vits16
        self.linear_layer=linear_layer
        self.transform = transform = pth_transforms.Compose([
            pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])
        
        self.vits16.eval()

    def forward(self, x, grad=True):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        if grad is False:
          with torch.no_grad():
            x = self.transform(x)
            
            # forward
            intermediate_output = self.vits16.get_intermediate_layers(x, 4)
            output = torch.cat([x[:, 0] for x in intermediate_output], dim=-1)
            output = linear_classifier(output)
            return output
        else:
          x = self.transform(x)
            
          # forward
          intermediate_output = self.vits16.get_intermediate_layers(x, 4)
          output = torch.cat([x[:, 0] for x in intermediate_output], dim=-1)
          output = linear_classifier(output)
          return output

In [59]:
model = ViTModel(vits16, linear_classifier)

# Adversarial training

In [60]:
# Define attack used for adversarial training
import torchattacks
from torchattacks import *
import torch.optim as optim

train_attack = PGD(model, eps=0.3, alpha=0.1, steps=12)

In [61]:
# Training configuration
NUM_EPOCHS = 5
model.linear_layer.train()
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.linear_layer.parameters(), lr=0.001)

In [ ]:
import tqdm

for epoch in range(NUM_EPOCHS):
    
    for i, (batch_images, batch_labels, _) in enumerate(train_loader):
        
        X = train_attack(batch_images, batch_labels).cuda()
        Y = batch_labels.cuda()

        predictions = model(X)
        cost = loss(predictions, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if i%25 == 0:
          print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], batch [{i+1}/{len(train_loader)}], Loss: {cost.item()}')

          # Test accuracy for clean samples
          correct_clean = 0
          correct_adv = 0

          for j, (imgs, labels, _) in enumerate(test_loader):
            imgs = imgs.to(device)
            labels = labels.to(device)
            predictions = model(imgs, False)
            correct_clean += torch.sum(torch.eq(predictions.argmax(1), labels))

            adv_samples = train_attack(imgs, labels).cuda()
            predictions = model(adv_samples, False)
            correct_adv += torch.sum(torch.eq(predictions.argmax(1), labels))

          del imgs
          del labels
          del adv_samples

          print(f"Clean accuracy [{correct_clean}/{len(val_set)}] = {correct_clean/len(val_set)}")
          print(f"Adversarial accuracy [{correct_adv}/{len(val_set)}] = {correct_adv/len(val_set)}\n")

Epoch [1/5], batch [1/100], Loss: 23.26812171936035
Clean accuracy [9/200] = 0.044999998062849045
Adversarial accuracy [0/200] = 0.0

Epoch [1/5], batch [26/100], Loss: 12.628471374511719
Clean accuracy [35/200] = 0.17499999701976776
Adversarial accuracy [0/200] = 0.0

Epoch [1/5], batch [51/100], Loss: 8.215069770812988
Clean accuracy [48/200] = 0.23999999463558197
Adversarial accuracy [0/200] = 0.0

Epoch [1/5], batch [76/100], Loss: 7.8168745040893555
Clean accuracy [37/200] = 0.1850000023841858
Adversarial accuracy [0/200] = 0.0

Epoch [2/5], batch [1/100], Loss: 8.419961929321289
Clean accuracy [36/200] = 0.17999999225139618
Adversarial accuracy [0/200] = 0.0

Epoch [2/5], batch [26/100], Loss: 8.056096076965332
